# Theoretical Questions

* This is the theoretical part of the final project. It includes theoretical questions from various topics covered in the course.
* There are 7 questions among which you need to choose 6, according to the following key:
    + Question 1 is **mandatory**.
    + Choose **one question** from questions 2-3.
    + Question 4 is **mandatory**.
    + Questions 5-6 are **mandatory**.
    + Question 7 is **mandatory**.
* Question 1 is worth 15 points, whereas the other questions worth 7 points.
* All in all, the maximal grade for this parts is 15+7*5=50 points.
* **You should answer the questions on your own. We will check for plagiarism.**
* If you need to add external images (such as graphs) to this notebook, please put them inside the 'imgs' folder. DO NOT put a reference to an external link.
* Good luck!

## Part 1: General understanding of the course material

### Question 1

1.  Relate the number of parameters in a neural network to the over-fitting phenomenon (*).
    Relate this to the design of convolutional neural networks, and explain why CNNs are a plausible choice for an hypothesis class for visual classification tasks.

    (*) In the context of classical under-fitting/over-fitting in machine learning models.

**Answer**

In machine learning, the number of parameters in a neural network plays a crucial role in determining its capacity to learn from the data and its susceptibility to overfitting. Overfitting occurs when a model becomes too complex (high parameter count) and starts to memorize noise or random fluctuations in the training data rather than generalizing patterns. In contrast, underfitting happens when a model is too simple (low parameter count) to capture the underlying patterns in the data.

If we were to use an architecture like MLP for visual classification tasks, we would create a model with a very large number parameters. For example, if we have a 1000x1000x3 color image, we would need about three million weights per neuron. As we saw in tutorial 3, huge number of parameters greatly increases the risk of overfitting. Another reason why MLP would be bad for visual classification tasks is that fully-connected layers are highly sensitive to translation, while image features are inherently translation-invariant. 

CNNs are a plausible choice for an hypothesis class for visual classification tasks for multiple reasons:
* Parameter Sharing: Neurons that are at the same depth in the grid share the same weights (parameters W, b). This significantly decreases the number of parameters in the model in comparison to MLPs.
* Translation Invariance: as we saw in tutorial 3, Convolution is a linear and shift-equivariant operator. The same features can appear in different parts of an image, and CNNs can detect them regardless of their location. This reduces the model's sensitivity to small changes in the position of features, making it more robust and less prone to overfitting.
* Pooling Layers: pooling layers typically reduce the spatial dimensions of the feature maps, reducing the risk of overfitting by focusing on the most salient features. This also allows us to learn a hierarchy of visual features. Pooling layers also add invariance to changes in the input.
* We can combine conv-conv-...-conv-pool blocks (learn to detect specific features in an image) with a FC-softmax block (extracts features for classification) to make a relatively powerful visual classification network (like VGG for example).




2. Consider the linear classifier model with hand-crafted features: 
    $$f_{w,b}(x) = w^T \psi(x) + b$$
    where $x \in \mathbb{R}^2$, $\psi$ is a non-learnable feature extractor and assume that the classification is done by $sign(f_{w,b}(x))$. Let $\psi$ be the following feature extractor $\psi(x)=x^TQx$ where $Q \in \mathbb{R}^{2 \times 2}$ is a non-learnable positive definite matrix. Describe a distribution of the data which the model is able to approximate, but the simple linear model fails to approximate (hint: first, try to describe the decision boundary of the above classifier).

**Answer**

<center><img src="imgs/Q1_2_a.png" /></center>
<center><img src="imgs/Q1_2_b.png" /></center>
<center><img src="imgs/Q1_2_c.png" /></center>


3. Assume that we would like to train a Neural Network for classifying images into $C$ classes. Assume that the architecture can be stored in the memory as a computational graph with $N$ nodes where the output is the logits (namely, before applying softmax) for the current batch ($f_w: B \times Ch \times H \times W \rightarrow B \times C$). Assume that the computational graph operates on *tensor* values.
    * Implement the CE loss assuming that the labels $y$ are hard labels given in a LongTensor of shape $B \times 1$. **Use Torch's log_softmax and gather functions** and implement with less as possible operations.

In [ ]:
from torch.nn.functional import log_softmax
from torch import gather
# Input:  model, x, y. 
# Output: the loss on the current batch.
logits = model(x)

# apply log_softmax on logits (along the rows):
logits_log_prob = log_softmax(logits, dim=1) 

# get the log probs for the true classes
selected_logits = -1 * gather(logits_log_prob, dim=1, index=y)

# average over batch as said in piazza post
loss = selected_logits.mean()

* Using the model's function as a black box, draw the computational graph (treating both log_softmax and gather as an atomic operations). How many nodes are there in the computational graph?

**Answer**

7 nodes in total.

<img src="imgs/Q1_3_b.png" />

* Now, instead of using hard labels, assume that the labels are representing some probability distribution over the $C$ classes. How would the gradient computation be affected? analyze the growth in the computational graph, memory and computation.

**Answer**

Note: we are assuming that the labels come as probability distribution, i.e. y is a tensor of shape (N,C) s.t. N is the batch size. In the hard case, y was of shape (N,1).
Let Q be logits_log_prob as defined in the code section above.
When we had hard labels, our CE loss calculation was very straightforward, thanks to the fact that y was a vector of 0s and one 1. Note that we use gather to find our desired label.

<img src="imgs/Q1_3_c.png" />

With the labels as a probability distribution, we will need to take the sum:

<img src="imgs/Q1_3_d.png" />

The computation graph for smooth labels will look like:

<img src="imgs/Q1_3_e.png" />

* The loss is computed in a different way, which takes longer because of the sum, approximately by a factor of C.
* The computational graph has a similar structure.
* When computing the gradient, we need to consider the gradient with respect to the probabilities for all classes, not just the true class.
* Instead of storing just the class label (one number), we will need to store C numbers (probability distribution) for each label y_n. 



* Apply the same analysis in the case that we would like to double the batch size. How should we change the learning rate of the optimizer?

**Answer**

* Doubling the batch size reduces the stochasticity of the gradient estimates because more samples are used to compute each gradient update. This can result in more stable gradient estimates and potentially lead to smoother convergence trajectories.

* On the other hand, doubling the batch size could also make the optimization landscape flatter, potentially causing the model to converge to a suboptimal solution or getting stuck in plateaus.

* Learning Rate Adjustment:
When you double the batch size, the effective batch size used for gradient updates becomes larger. This means that the gradients are computed with more samples, which can lead to larger gradient magnitudes. As a result, you might need to adjust the learning rate to accommodate the change in batch size.

According to https://arxiv.org/abs/1404.5997, the learning rate should be adjusted inversely proportional to the square root of the batch size change. The reasoning behind this is that when you double the batch size, the gradient magnitude scales proportionally with the batch size. To maintain a similar update magnitude and convergence behavior, you would want to increase the learning rate. The specific scaling factor is the square root of the ratio of new batch size to old batch size. In our case the new learning rate is equal to sqrt(2) times the old learning rate.


## Part 2: Optimization & Automatic Differentiation

### Question 2: resolving gradient conflicts in multi-task learning

Assume that you want to train a model to perform two tasks: task 1 and task 2. 
For each such task $i$ you have an already implemented function *loss\_i = forward_and_compute_loss_i(model,inputs)* such that given the model and the inputs it computes the loss w.r.t task $i$ (assume that the computational graph is properly constructed). We would like to train our model using SGD to succeed in both tasks as follows: in each training iteration (batch) -
* Let $g_i$ be the gradient w.r.t the $i$-th task.
* If $g_1 \cdot g_2 < 0$:
    + Pick a task $i$ at random.
    + Apply GD w.r.t only that task.
* Otherwise:
    + Apply GD w.r.t both tasks (namely $\mathcal{L}_1 + \mathcal{L}_2$).

Note that in the above formulation the gradient is a thought of as a concatination of all the gradient w.r.t all the models parameters, and $g_1 \cdot g_2$ stands for a dot product.

What parts should be modified to implement the above? Is it the optimizer, the training loop or both? Implement the above algorithm in a code cell/s below

**Answer**

We chose to implement Question 3 instead of Question 2.

### Question 3: manual automatic differentiation

Consider the following two-input two-output function:
$$ f(x,y) = (x^2\sin(xy+\frac{\pi}{2}), x^2\ln(1+xy)) $$
* Draw a computational graph for the above function. Assume that the unary atomic units are squaring, taking square root, $\exp,\ln$, basic trigonometric functions and the binary atomic units are addition and multiplication. You would have to use constant nodes.
* Calculate manually the forward pass.
* Calculate manually the derivative of all outputs w.r.t all inputs using a forward mode AD.
* Calculate manually the derivative of all outputs w.r.t all inputs using a backward mode AD.

**Answer** 

**Computational graph for the above function**: 

<img src="imgs/Q3_atest.png" />

**forward pass with point (x0, y0)**

<img src="imgs/Q3_b.png" />

**Derivative of all outputs using a forward mode automatic differentiation:** 

with respect to x:

<img src="imgs/Q3_c.png" />


with respect to y:

<img src="imgs/Q3_d.png" />

**Derivative of all outputs using a backward mode automatic differentiation:** 

Note: a piazza post was released that states we can use whatever point we would like, so we chose to use point (1,0). This was our work before the piazza post was released when we thought we needed to use a general point.

<center><img src="imgs/Q3_comp_graph.png" /></center>

<center><img src="imgs/Q3_backward_1.png" /></center>

<center><img src="imgs/Q3_backward_2.png" /></center>

## Part 3: Sequential Models

### Question 4: RNNs vs Transformers in the real life

In each one of the following scenarios decide whether to use RNN based model or a transformer based model. Justify your choice.
1. You are running a start-up in the area of automatic summarization of academic papers. The inference of the model is done on the server side, and it is very important for it to be fast.
2. You need to design a mobile application that gathers small amount of data from few apps in every second and then uses a NN to possibly generate an alert given the information in the current second and the information from the past minute.
3. You have a prediction task over fixed length sequences on which you know the following properties:
    + In each sequence there are only few tokens that the model should attend to.
    + Most of the information needed for generating a reliable prediction is located at the beginning of the sequence.
    + There is no restriction on the computational resources.

**Answer**

**4.1)** We believe it to be better to use a transformer instead of an RNN. This is because transformers are able to process tokens in parallel, making them more efficient in terms of hardware utilization compared to RNNs, which process sequences sequentially. This parallelization capability can significantly speed up inference times. 

Additionally, academic papers can be quite long, and transformers have shown to be better at capturing long-range dependencies in text compared to RNNs. This is particularly relevant for summarization tasks, where the model needs to consider information from different parts of the paper to generate an effective summary.

Moreover,  transformers use self-attention mechanisms that allow the model to focus on relevant parts of the input sequence. This attention mechanism enables better comprehension of the context and can lead to more informative summaries.

Finally, pretrained transformer models, such as BERT and GPT, are widely available, which allows us to leverage transfer learning. This means we can fine-tune a pretrained transformer on our specific summarization task, saving both training time and resources.


**4.2)**  In this case, we should use an RNN instead of a transformer. This is because RNNs are designed to handle sequential data, making them well-suited for time-series analysis. In our case, the data collected from different apps every second represents a time series, and an RNN can effectively model the temporal dependencies between these sequential data points.

Additionally, in real-time applications like this, RNNs can be adapted to an online learning setting where we can update the model continuously as new data becomes available. This allows the model to adapt to changing patterns in the data over time.

Moreover, we know that RNNs have a memory-like structure that allows them to maintain information about past inputs, which is crucial for making decisions based on both the current second's data and the information from the past minute.

Finally, compared to transformer models, RNNs typically have lower computational overhead, which is important for mobile applications with limited processing power and memory.


**4.3)** In this case, we should use a transformer instead of an RNN. This is because transformers are well-suited for tasks where only a few tokens are important because of their attention mechanism. The self-attention mechanism in transformers allows the model to focus on relevant tokens and disregard irrelevant ones, which can be beneficial for scenarios with sparse dependencies. RNNs on the other hand, do not handle these sparse dependency scenarios as efficiently.

Additionally, transformers can capture long-range dependencies in the sequence effectively. Since most of the important information is located at the beginning of the sequence, the transformer can efficiently attend to those tokens and leverage them for the prediction task. RNNs, on the other hand, are often less efficient at maintaining the importance of tokens found nearer the beginning of the sequence. 

Finally, although there are no restrictions on computational resources, transformers inherently support parallelization, which can still be advantageous when dealing with large amounts of data or when optimizing training time.



## Part 4: Generative modeling

### Question 5: VAEs and GANS

Suggest a method for combining VAEs and GANs. Focus on the different components of the model and how to train them jointly (the objectives). Which drawbacks of these models the combined model may overcome? which not?

**Answer**

The aim of this combining VAE and GAN is to leverage the strengths of both models to create a more robust and expressive generative model. The primary components of the combined model are the encoder and decoder from VAEs and the discriminator from GANs. Below is an explanation of the model and its training:

1. Architecture:
   - Encoder: The encoder takes an input data point and maps it to a latent space representation, just like in a VAE.
   - Decoder: The decoder takes a point from the latent space and generates a reconstructed output, again similar to a VAE.
   - Discriminator: The discriminator, like in a GAN, is responsible for distinguishing between real data samples and fake (generated) data samples.

2. Objective Functions:
   - Reconstruction Loss (VAE part): This is the same as in a traditional VAE and aims to minimize the difference between the input data and the reconstructed output.
   - Adversarial Loss (GAN part): The discriminator tries to distinguish between real data and the reconstructed data from the decoder, while the decoder aims to generate data that can "fool" the discriminator.
   - Latent Regularization (VAE part): The VAE's objective is to make the latent space as close to a standard Gaussian distribution as possible, enforcing the learned features to be more structured and interpretable.

3. Joint Training:
   The combined model is trained jointly, and the objectives are optimized simultaneously:
   - During the training of the encoder-decoder (VAE) part, the reconstruction loss and latent regularization loss are computed and minimized to improve the quality of the generated data.
   - During the training of the discriminator (GAN) part, the adversarial loss is computed to enhance the discriminator's ability to distinguish between real and generated samples.
   - The encoder and decoder are updated to minimize the reconstruction loss and latent regularization loss while considering the gradients from the discriminator to improve the generated samples' quality.

4. Benefits and Drawbacks:
   - Benefits:
     - Adversarial training can enhance the quality and diversity of the generated samples, addressing some of the mode collapse issues that GANs may face. 
     - GANs are very tough to train, due to mode collapse. The structured latent space generated by the VAE might potentially lead to more stable GAN training.
     - By incorporating the VAE's latent space regularization, the combined model may achieve better disentangled and interpretable feature representations.
     - The model can learn a more structured and probabilistic representation of data, allowing for meaningful interpolation in the latent space.
     - A major drawback of VAEs is that their generated outputs are often blurry. The discriminator will select for sharper images, which will potentially mitigate the blurriness of the generated VAE images.

   - Drawbacks:
     - Training can be more complex and computationally intensive due to the combination of two models with different objectives.
     - The model may still suffer from mode collapse, though it might be less severe compared to standalone GANs.
     - Balancing the different components' losses can be challenging and may require hyperparameter tuning, and VAE’s and GAN’s sensitivity to hyperparameters is a major problem.

The combination of VAEs and GANs provides a powerful framework that addresses some of the limitations of both models, leading to improved sample quality, better disentangled representations, and more stable training. 



### Question 6: Diffusion Models

Show that $q(x_{t-1}|x_t,x_0)$ is tractable and is given by $\mathcal{N}(x_{t-1};\tilde{\mu}(x_t,x_0),\tilde{\beta_t}I)$ where the terms for $\tilde{\mu}(x_t,x_0)$ and $\tilde{\beta_t}$ are given in the last tutorial. Do so by explicitly computing the PDF.

**Answer:**

<center><img src="imgs/Q6_1.png" /></center>

<center><img src="imgs/Q6_2.png" /></center>

<center><img src="imgs/Q6_3.png" /></center>

<center><img src="imgs/Q6_4.png" /></center>

<center><img src="imgs/Q6_5.png" /></center>

<center><img src="imgs/Q6_6.png" /></center>

<center><img src="imgs/Q6_7.png" /></center>

## Part 5: Training Methods

### Question 7: Batch Normalization and Dropout

For both BatchNorm and Dropout analyze the following:
1. How to use them during the training phase (both in forward pass and backward pass)?
2. How differently they behave in the inference phase? How to distinguish these operation modes in code?
3. Assume you would like to perform multi-GPU training (*) to train your model. What should be done in order for BatchNorm and dropout to work properly? assume that each process holds its own copy of the model and that the processes can share information with each other.

(*): In a multi-GPU training each GPU is associated with its own process that holds an independent copy of the model. In each training iteration a (large) batch is split among these processes (GPUs) which compute the gradients of the loss w.r.t the relevant split of the data. Afterwards, the gradients from each process are then shared and averaged so that the GD would take into account the correct gradient and to assure synchornization of the model copies. Note that the proccesses are blocked between training iterations.

**Answer**

1. Training Phase - Forward Pass and Backward Pass:

BatchNorm (Batch Normalization):
- During the forward pass: BatchNorm normalizes the activations of each layer in a mini-batch, typically performed after the linear transformation and before the activation function. It centers and scales the activations, making the network more robust and accelerating convergence.
- During the backward pass: BatchNorm computes gradients with respect to the input data, scale, and shift parameters of the BatchNorm layer. These gradients are used to update the parameters during the optimization step.

Dropout:
- During the forward pass: Dropout randomly sets a fraction of the units in a layer to zero, effectively "dropping them out" temporarily. This is done to prevent overfitting and encourages the network to learn more robust features. We also scale the remaining units.
- During the backward pass: During backpropagation, the gradients are only propagated through the non-dropped units. The dropped units have no effect on the gradients and are effectively ignored. The gradients are also scaled.

2. Inference Phase:

BatchNorm:
- During inference, BatchNorm does not rely on the statistics of the mini-batch, as there is no batch to compute statistics on.
- Instead, during inference, BatchNorm uses the population statistics (mean and variance) learned during training to normalize the activations.

Dropout:
-During inference, Dropout is usually turned off, as it is primarily a regularization technique to prevent overfitting during training.
- In some cases, people use a scaled-down version of the model where the weights of the dropped-out units are scaled by the dropout probability during inference.

Distinguishing operation modes in code:
During training, we would call model.train() to set to training phase.
We would call model.eval() to set the model to inference phase. Dropout layers don’t do anything, and BatchNorm layers use running statistics instead of batch statistics.

3. Multi-GPU Training:

BatchNorm:
- We will need a process that will aggregate the data sent to it by the GPUs, we will call it agg-p.
- Each GPU calculates the mean and variance of the batch it was assigned. They will then send this statistics to agg-p.
- agg-p will now compute the global mean and the global variance. It will send these statistics back to every GPU. Now every GPU uses the same statistics for its normalization.
- After the backpropagation step, the gradients are averaged together. 


Dropout:
- We will let one main GPU generate a dropout mask. It will then share this mask with all of the other GPUs.
- The GPUs proceed to do their forward and backward steps. Because of one shared mask, there is uniformity across which nodes are dropped out and which are not.
- At the end of the backward step, the computed gradients of the GPUs are averaged together.

